# 프로젝트 - Movielens 영화 추천 실습

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
* 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
* 별점을 시청횟수로 해석해서 생각하겠습니다.
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
         
Cloud Storage에 미리 업로드된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결해 줍니다.

## 1) 데이터 준비와 전처리

Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.


In [1]:
import os
import pandas as pd
import seaborn as sns
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_names = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_names, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남긴다. 
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꾼다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 로드
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### user정보와 movie정보의 고유값(key)인 moivie_id을 기준으로 병합

In [6]:
data = pd.merge(ratings ,movies)
data.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


### 사용하는 컬럼만 남겨줍니다. (drop으로도 가능)

In [7]:
using_cols = ['user_id', 'title', 'counts']
data = data[using_cols]
data.head(10)

,user_id,title,counts
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2,One Flew Over the Cuckoo's Nest (1975),5
2,12,One Flew Over the Cuckoo's Nest (1975),4
3,15,One Flew Over the Cuckoo's Nest (1975),4
4,17,One Flew Over the Cuckoo's Nest (1975),5
5,18,One Flew Over the Cuckoo's Nest (1975),4
6,19,One Flew Over the Cuckoo's Nest (1975),5
7,24,One Flew Over the Cuckoo's Nest (1975),5
8,28,One Flew Over the Cuckoo's Nest (1975),3
9,33,One Flew Over the Cuckoo's Nest (1975),5


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

## 2) 분석해 봅시다.
* ratings에 있는 유니크한 영화 개수
* ratings에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

nunique 은 중복 제거하고 데이터 갯수 보여줌  
unique 은 어떤 데이터 값들이 있는지를 중복 제거하고 보여줌

In [8]:
# data에 있는 유니크한 영화 개수
data['title'].nunique()

3628

In [9]:
# data에 있는 유니크한 사용자 수
data['user_id'].nunique()

6039

In [10]:
# 가장 인기 있는 영화 30개(인기순)
title_count = data.groupby('title')['user_id'].count()
title_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [11]:
# 사용자 별 몇 개의 영화를 봤는지에 대한 통계
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

In [12]:
# 그래프로 확인

#sns.barplot(x = data.groupby('user_id')['title'].count())

## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [13]:
my_favorite = ['Terminator 2: Judgment Day (1991)' , 'Silence of the Lambs, The (1991)   ' ,'Matrix, The (1999)' ,'Sixth Sense, The (1999)' ,'Men in Black (1997)']

# 'changmin'이라는 user_id가 위 영화를 3회씩 들었다고 가정하겠습니다.
my_movielist = pd.DataFrame({'user_id': ['changmin']*5, 'title': my_favorite, 'counts':[3]*5})

if not data.isin({'user_id':['changmin']})['user_id'].any():  # user_id에 'changmin'이라는 데이터가 없다면
    data = data.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,title,counts
836473,5851,One Little Indian (1973),5
836474,5854,Slaughterhouse (1987),4
836475,5854,"Promise, The (Versprechen, Das) (1994)",3
836476,5938,"Five Wives, Three Secretaries and Me (1998)",4
836477,5948,Identification of a Woman (Identificazione di ...,5
0,changmin,Terminator 2: Judgment Day (1991),3
1,changmin,"Silence of the Lambs, The (1991)",3
2,changmin,"Matrix, The (1999)",3
3,changmin,"Sixth Sense, The (1999)",3
4,changmin,Men in Black (1997),3


### 모델에 활용하기 위한 전처리 (실습)

사람이 태어나면 주민등록번호가, 학교에 가면 출석번호가 있듯이 데이터의 관리를 쉽게 하기 위해 번호를 붙여주고 싶습니다. 우리가 다루는 데이터에서는 user와 artist 각각에 번호를 붙이고 싶습니다. 보통 이런 작업을 indexing이라고 합니다. 추천 시스템, 자연어 처리에서 자주 하는 작업들이라 직접 해보시면 좋겠습니다.

In [14]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 영화 indexing하는 코드
user_to_idx = {v:k for k, v in enumerate(user_unique)}
title_to_idx = {v:k for k, v in enumerate(title_unique)}

In [15]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['changmin'])  # 6040명의 유저 중 마지막으로 추가된 유저라서 6039이 나와야 한다. 
print(title_to_idx['Matrix, The (1999)'])

6039
124


In [16]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 참고

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해보자. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될테니 dropna()로 제거
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):  # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data
else:
    print('user_id column indexing Fail!!')
    
# title_to_idx를 통해 artist 컬럼도 동일한 방식으로 인덱싱
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,counts
0,0,0,5
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,5
...,...,...,...
0,6039,92,3
1,6039,3628,3
2,6039,124,3
3,6039,38,3


data의 user_id와 artist 컬럼 내 값들이 모두 정수 인덱스 값으로 잘 변경었습니다. 이것으로 훈련을 위한 전처리가 완료되었습니다.

## 4) CSR(Compressed Sparse Row) Matrix를 직접 만들어 봅시다.

In [17]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape=(num_user, num_title))
csr_data

<6040x3629 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

우리가 앞으로 만들어갈 모델에서는 암묵적 데이터의 해석을 위해 다음과 같은 규칙을 적용할 것입니다.

한 번이라도 들었으면 선호한다고 판단한다.
많이 재생한 아티스트에 대해 가중치를 주어서 더 확실히 좋아한다고 판단한다.

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

### MF 모델 학습하기

* implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지입니다.  
* 이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용하겠습니다. Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있습니다.

In [18]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보겠습니다.

* factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
* regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
* use_gpu : GPU를 사용할 것인지
* iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지  

1과 4를 늘릴수록 학습 데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [19]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [20]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3629x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [21]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

### 모델 학습이 끝났습니다 이제 2가지를 아래 사항을 살펴보도록 하겠습니다.

1. changmin 벡터와 Matrix, The (1999)의 벡터를 어떻게 만들고 있는지
2. 두 벡터를 곱하면 어떤 값이 나오는지


In [22]:
changmin, Matrix_The_1999 = user_to_idx['changmin'], title_to_idx['Matrix, The (1999)']
changmin_vector, Matrix_The_1999_vector = als_model.user_factors[changmin], als_model.item_factors[Matrix_The_1999]
print('슝=3')

슝=3


In [23]:
changmin_vector

array([-0.17914243,  0.326262  ,  0.18955071,  0.47753876, -0.36719096,
        0.07101326, -0.47124076, -0.32982677, -0.34534726, -0.20842533,
       -0.1661417 , -0.05393067, -0.11867929, -0.61595196,  0.40284127,
       -0.04195413, -0.7882799 , -0.21388821,  0.5389449 ,  0.04755956,
        0.04157804,  0.0116101 ,  0.24355316,  0.34569326, -0.3104159 ,
       -0.28699216,  0.26446617, -0.0578205 ,  0.06816526,  0.8625837 ,
        0.37200218,  0.06526683,  0.36259845, -0.46804917,  0.26409134,
        0.40273827,  0.28691256,  0.18363477, -0.56123865, -0.10990005,
        0.01719218, -0.400725  , -0.01218794,  0.4724239 ,  0.24406435,
       -0.31994686, -0.21682104,  0.28429025, -0.297543  ,  0.03616602,
        0.32581314,  0.4224097 , -0.8138117 , -0.04328337, -0.06695866,
        0.59934014,  0.49767795, -0.667124  , -0.5743107 ,  1.0210266 ,
       -0.35531247, -0.86971825, -0.31047744, -0.05362161, -0.12664475,
       -0.12344241,  0.66187644,  0.35977155, -0.23090336, -0.15

In [24]:
Matrix_The_1999_vector

array([-0.00021273,  0.01587959,  0.0172383 ,  0.02782312, -0.00501156,
       -0.00763333, -0.02038139, -0.01950873, -0.01620864, -0.00652421,
       -0.01413937,  0.00742623, -0.0096157 , -0.00889781,  0.00512385,
        0.03467052, -0.02081076,  0.00824839,  0.01727368, -0.00937213,
        0.00729305,  0.01448663,  0.01790546,  0.00817062, -0.00093605,
        0.00440212,  0.01031222,  0.00801695,  0.00058064,  0.0408287 ,
        0.01985225,  0.02631734,  0.01571124, -0.01400559,  0.01818045,
       -0.00644386,  0.03520104,  0.009517  ,  0.00304847,  0.00052352,
        0.01908033, -0.00795843,  0.02125354,  0.01949961,  0.02907239,
       -0.01135551, -0.00758229,  0.03086883, -0.00827681,  0.01519198,
        0.01878341,  0.02124492, -0.01403974,  0.00562721, -0.00302977,
        0.02259221,  0.02712061,  0.00750748, -0.01699543,  0.04178003,
       -0.00143391, -0.02017764, -0.0158458 ,  0.02166191, -0.00955881,
       -0.00833225,  0.040587  ,  0.01468922, -0.00702858, -0.00

In [25]:
# changmin과 Matrix_The_1999_vector를 내적하는 코드
np.dot(changmin_vector,Matrix_The_1999_vector)

0.51334596

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 영화를 찾습니다. 처음으로는 제가 좋아하는 Matrix, The (1999)로 찾아보겠습니다.

In [26]:
favorite_title = 'Matrix, The (1999)'
title_id = title_to_idx[favorite_title]
similar_title = als_model.similar_items(title_id, N=15)
similar_title

[(124, 1.0000001),
 (92, 0.7754812),
 (62, 0.6699194),
 (200, 0.59620863),
 (141, 0.57743853),
 (145, 0.57103515),
 (375, 0.54152644),
 (107, 0.5015219),
 (317, 0.49594423),
 (175, 0.4815462),
 (75, 0.46178716),
 (3628, 0.4366822),
 (44, 0.4316521),
 (236, 0.40742457),
 (117, 0.4018374)]

(영화 id, 유사도) Tuple 로 반환하고 있습니다. 영화의 id를 다시 영화의 이름으로 매핑 시켜 주겠습니다.

In [27]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_title]

['Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Terminator, The (1984)',
 'Fugitive, The (1993)',
 'Fifth Element, The (1997)',
 'Face/Off (1997)',
 'Jurassic Park (1993)',
 'Twelve Monkeys (1995)',
 'Men in Black (1997)',
 'Hunt for Red October, The (1990)',
 'Silence of the Lambs, The (1991)   ',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Speed (1994)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)']

SF,액션물

In [28]:
# 몇 번 더 반복해서 확인하기 위해 위의 코드 함수화
def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [29]:
get_similar_title('Terminator 2: Judgment Day (1991)')

['Terminator 2: Judgment Day (1991)',
 'Matrix, The (1999)',
 'Total Recall (1990)',
 'Jurassic Park (1993)',
 'Terminator, The (1984)',
 'Men in Black (1997)',
 'Fugitive, The (1993)',
 'Braveheart (1995)',
 'Hunt for Red October, The (1990)',
 'Silence of the Lambs, The (1991)   ']

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

* AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 제가 좋아할 만한 영화를 추천받습니다.  
* filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [30]:
user = user_to_idx['changmin']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
title_recommended

[(107, 0.48648062),
 (62, 0.3562358),
 (60, 0.32124487),
 (141, 0.2731817),
 (200, 0.25828737),
 (75, 0.22319573),
 (121, 0.22241783),
 (670, 0.2203012),
 (236, 0.20305148),
 (87, 0.2011298),
 (99, 0.19584593),
 (145, 0.1940937),
 (375, 0.19324929),
 (64, 0.1889112),
 (44, 0.18241107),
 (82, 0.17498057),
 (317, 0.16893777),
 (220, 0.16726539),
 (233, 0.16315898),
 (51, 0.15992509)]

In [31]:
[idx_to_title[i[0]] for i in title_recommended]

['Jurassic Park (1993)',
 'Total Recall (1990)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Fugitive, The (1993)',
 'Terminator, The (1984)',
 'Hunt for Red October, The (1990)',
 'Silence of the Lambs, The (1991)',
 'Galaxy Quest (1999)',
 'Speed (1994)',
 'Braveheart (1995)',
 'American Beauty (1999)',
 'Fifth Element, The (1997)',
 'Face/Off (1997)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Lost World: Jurassic Park, The (1997)',
 'Twelve Monkeys (1995)',
 'Seven (Se7en) (1995)',
 'Usual Suspects, The (1995)',
 'Fargo (1996)']

In [32]:
Speed_1994 = title_to_idx['Speed (1994)']
explain = als_model.explain(user, csr_data, itemid=Speed_1994)

In [33]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Terminator 2: Judgment Day (1991)', 0.06359672042064606),
 ('Matrix, The (1999)', 0.06250124272624165),
 ('Sixth Sense, The (1999)', 0.039702327159538545),
 ('Men in Black (1997)', 0.033405467482217815),
 ('Silence of the Lambs, The (1991)   ', 4.8249613291653876e-05)]